# Hand Protection System for Shredder - Model Integration

This notebook integrates the SSD MobileNet model and other utility scripts for real-time hand detection and safety alerts.

## 1. Setup and Installation

Install all required libraries and ensure TensorFlow 2.x is available.

In [1]:
!conda activate shredder

In [2]:
!pip install -r requirements.txt

  Using cached absl-py-0.7.1.tar.gz (99 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [6 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\lenin\AppData\Local\Temp\pip-install-cc7ijhz_\absl-py_3509d4fbe6134d5a8f0260b5fe4cd3b3\setup.py", line 34, in <module>
      raise RuntimeError('Python version 2.7 or 3.4+ is required.')
  RuntimeError: Python version 2.7 or 3.4+ is required.
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


## 2. Load Pre-trained SSD MobileNet Model

Load the frozen inference graph, label map, and pipeline configuration.

In [11]:

import tensorflow as tf
from utils import label_map_util

# Paths to the model and label map
frozen_graph_path = r"C:\Users\lenin\Downloads\New folder\frozen_graphs\ssd5_optimized_inference_graph.pb"  # Replace with the correct frozen graph
label_map_path = r"C:\Users\lenin\Downloads\New folder\frozen_graphs\Glove_label_map.pbtxt"  # Replace with the correct label map

# Load the model
def load_model(frozen_graph_path):
    print("Loading the frozen inference graph...")
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.compat.v1.GraphDef()
        with tf.io.gfile.GFile(frozen_graph_path, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return detection_graph

detection_graph = load_model(frozen_graph_path)
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=2, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print("Model and label map loaded successfully.")


Loading the frozen inference graph...


NotFoundError: NewRandomAccessFile failed to Create/Open: C:\Users\lenin\Downloads\New folder\frozen_graphs\ssd5_optimized_inference_graph.pb : The system cannot find the path specified.
; No such process

## 3. Safety Lines and Alerts

Draw safety lines and integrate alert mechanisms for boundary violations.

In [ ]:

import cv2
from utils import orien_lines, alertcheck

def display_safety_lines(image, orientation="bt", perc1=15, perc2=30):
    line_pos = orien_lines.drawsafelines(image, orientation, perc1, perc2)
    return image, line_pos

def process_alerts(image, p1, p2, line_position, orientation):
    alert_triggered = alertcheck.drawboxtosafeline(image, p1, p2, line_position, orientation)
    return alert_triggered


## 4. Real-Time Detection

Run the SSD MobileNet model for real-time hand detection, display safety lines, and trigger alerts if necessary.

In [ ]:

def real_time_detection():
    score_thresh = 0.5
    video_stream = cv2.VideoCapture(0)

    with detection_graph.as_default():
        with tf.compat.v1.Session(graph=detection_graph) as sess:
            while True:
                ret, frame = video_stream.read()
                if not ret:
                    break

                # Preprocess image
                im_height, im_width, _ = frame.shape
                image_np_expanded = np.expand_dims(frame, axis=0)

                # Get tensors
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
                detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

                # Run detection
                (boxes, scores, classes) = sess.run(
                    [detection_boxes, detection_scores, detection_classes],
                    feed_dict={image_tensor: image_np_expanded})

                # Draw safety lines
                frame, line_position = display_safety_lines(frame, "bt")

                # Draw boxes and process alerts
                for i in range(boxes.shape[1]):
                    if scores[0][i] > score_thresh:
                        box = boxes[0][i]
                        ymin, xmin, ymax, xmax = box
                        p1 = (int(xmin * im_width), int(ymin * im_height))
                        p2 = (int(xmax * im_width), int(ymax * im_height))
                        process_alerts(frame, p1, p2, line_position, "bt")
                        cv2.rectangle(frame, p1, p2, (0, 255, 0), 2)

                # Display frame
                cv2.imshow("Real-Time Detection", frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break

    video_stream.release()
    cv2.destroyAllWindows()

real_time_detection()


In [ ]:
import os
import pandas as pd
from datetime import datetime

def save_hand_detection_data(no_of_time_hand_detected, no_of_time_hand_crossed, file_path='result.xlsx'):
    """Save hand detection data to an Excel file."""
    today = datetime.now().strftime('%Y-%m-%d')

    # Check if the file exists
    if os.path.exists(file_path):
        # Load existing data
        df = pd.read_excel(file_path)
        if today in df['Date'].values:
            # Update the existing row
            df.loc[df['Date'] == today, 'Number of times hand detected'] += no_of_time_hand_detected
            df.loc[df['Date'] == today, 'Number of times hand crossed'] += no_of_time_hand_crossed
        else:
            # Append a new row
            new_row = {
                'Date': today,
                'Number of times hand detected': no_of_time_hand_detected,
                'Number of times hand crossed': no_of_time_hand_crossed
            }
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    else:
        # Create a new DataFrame if the file doesn't exist
        df = pd.DataFrame({
            'Date': [today],
            'Number of times hand detected': [no_of_time_hand_detected],
            'Number of times hand crossed': [no_of_time_hand_crossed]
        })

    # Save the data back to the file
    df.to_excel(file_path, index=False)
    print(f"Data saved successfully to {file_path}")


In [ ]:
def real_time_detection():
    score_thresh = 0.5
    video_stream = cv2.VideoCapture(0)

    if not video_stream.isOpened():
        print("Error: Cannot access the camera.")
        return

    # Counters for detections and crossings
    lst1 = []
    lst2 = []

    while True:
        ret, frame = video_stream.read()
        if not ret:
            print("Error: Cannot read from camera.")
            break

        # Perform detection
        input_tensor = tf.convert_to_tensor(frame)
        input_tensor = input_tensor[tf.newaxis, ...]
        detections = detection_model(input_tensor)

        # Process detection results
        boxes = detections['detection_boxes'][0].numpy()
        scores = detections['detection_scores'][0].numpy()
        classes = detections['detection_classes'][0].numpy().astype(int)

        # Draw detections and safety lines
        frame, line_position = display_safety_lines(frame, "bt")
        for i, box in enumerate(boxes):
            if scores[i] > score_thresh:
                ymin, xmin, ymax, xmax = box
                p1 = (int(xmin * frame.shape[1]), int(ymin * frame.shape[0]))
                p2 = (int(xmax * frame.shape[1]), int(ymax * frame.shape[0]))
                detection_result = process_alerts(frame, p1, p2, line_position, "bt")
                lst1.append(detection_result)  # Track hand crossing
                lst2.append(1)  # Track hand detection
                cv2.rectangle(frame, p1, p2, (0, 255, 0), 2)

        # Display frame
        cv2.imshow("Real-Time Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    video_stream.release()
    cv2.destroyAllWindows()

    # Calculate hand detection and crossings
    def count_occurrences(lst):
        return sum(1 for i, val in enumerate(lst) if val == 1 and (i == 0 or lst[i - 1] == 0))

    no_of_time_hand_detected = count_occurrences(lst2)
    no_of_time_hand_crossed = count_occurrences(lst1)

    # Save data to Excel
    save_hand_detection_data(no_of_time_hand_detected, no_of_time_hand_crossed)
    print("Data saved successfully!")


In [25]:
!python hand_detection.py

2024-12-01 21:28:16.960441: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-01 21:28:18.769332: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Traceback (most recent call last):
  File "C:\Users\lenin\Downloads\New _folder\hand_detection.py", line 6, in <module>
    from utils import detector_utils as detector_utils
  File "C:\Users\lenin\Downloads\New _folder\utils\detector_utils.py", line 23, in <module>
    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lenin\

In [5]:
!.\venv\Scripts\activate
!pip install ipykernel

In [7]:
!python -m ipykernel install --user --name=shredder --display-name "shredder"


Installed kernelspec shredder in C:\Users\lenin\AppData\Roaming\jupyter\kernels\shredder


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [3]:
%run hand_detection.py

AttributeError: module 'tensorflow' has no attribute 'gfile'

In [13]:
!.\venv\Scripts\activate


In [15]:
!pip install tensorflow==1.15


ERROR: Could not find a version that satisfies the requirement tensorflow==1.15 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==1.15
